# (0) Download Butterfly 

In [ ]:
!kaggle datasets download -d gpiosenka/butterfly-images40-species

In [ ]:
import zipfile
with zipfile.ZipFile('butterfly-images40-species.zip', 'r') as zip_ref:
    zip_ref.extractall('data/butterfly')

# (1) Process Butterfly Data

In [2]:
from utils.data import create_butterfly_dataset
trainset, testset, normal_mapping, reverse_mapping = create_butterfly_dataset()
assert len(trainset) == 12594, 'Size of train set not match'
assert len(testset) == 500, 'Size of test set not match'

# (2) Import Classifier

In [3]:
from model.butterfly_classifier import DenseNet121
import torch

model = DenseNet121(num_classes=len(normal_mapping)).to('cuda')
model.load_state_dict(torch.load('./model/states/butterfly_classifier.pth'))

<All keys matched successfully>

# (3) Evaluate Untargeted Adversarial Examples

In [ ]:
from torch.utils.data import DataLoader
from utils.base import eval_accuracy


acc = eval_accuracy(model, testset,  batch_size=100)
print('Accuracy on test set is {}'.format(acc))

In [ ]:
from algo.attacker import adversarial_generator

test_img, test_label = testset[5]
test_img = test_img.unsqueeze(0)

adv_img = adversarial_generator(model = model, target_class=test_label,
                             image=test_img, 
                             search_var=1e-3,
                             sample_num=50,
                             bound=0.1,
                             lr=0.01,
                             query_limit=3000)

In [ ]:
import torch.nn.functional as F

adv_logits = model(adv_img)
org_logits = model(test_img.to('cuda'))

print('Adversarial: predicted class is {}'.format(torch.argmax(adv_logits, dim=1)))
print('Original: predicted class is {}'.format(torch.argmax(org_logits, dim=1)))

print('Adversarial: logit of true class is {}'.format(adv_logits[0, test_label]))
print('Original: logit of true class is {}'.format(org_logits[0, test_label]))

print('Adversarial: probability of true class is {}'.format(F.softmax(adv_logits, dim=1)[0, test_label]))
print('Original: probability of true class is {}'.format(F.softmax(org_logits, dim=1)[0, test_label]))

In [14]:
from tqdm import tqdm 
import torch
import torch.nn.functional as F
from torch import nn
def NES(model, target_class, image, search_var, sample_num, g, u):
    #parameters
    n = sample_num #should be even
    N = image.size(2) #assume the image is N x N may subject to change
    
    #NES estimation
    g.zero_()
    with torch.no_grad():
        for i in range(n):
            u.normal_()
            g = g + F.softmax(model(image + search_var * u), dim =1)[0,target_class] * u
            g = g - F.softmax(model(image - search_var * u), dim =1)[0,target_class] * u #we assume the output of the model is ordered by class index
    return 1 / (2*n*search_var) * g

def PIA_adversarial_generator(model, initial_image, image, target_class, epsilon_adv, epsilon_0, search_var, sample_num, delta_epsilon, eta_max, eta_min, k=5):
    device = next(model.parameters()).device
    initial_image, image = initial_image.to(device), image.to(device)
    x_adv = image.clone()
    N = initial_image.size(2)
    g = torch.zeros(N, requires_grad=False).to(device)
    u = torch.randn((N, N)).to(device)
    
    epsilon = epsilon_0
    x_adv = torch.clamp(x_adv, initial_image - epsilon, initial_image + epsilon)
    original_class = torch.argmax(model(initial_image), dim=1)
    print(original_class==69)
    
    with torch.no_grad():
        probabilities_adv = F.softmax(model(x_adv), dim=1)
        top_probs, top_classes = torch.topk(probabilities_adv, k)
        time = 0
        while (epsilon > epsilon_adv) | (original_class != target_class):
            if time%20 == 1:
                print("20 times runned")
                print(F.softmax(model(x_adv), dim=1))
                print(torch.argmax(model(x_adv)))
                print(epsilon)
            time += 1
            gradient = NES(model, target_class, x_adv, search_var, sample_num, g, u)
            eta = eta_max
            x_adv_hat = x_adv - eta * gradient
            
            while not target_class in top_classes[0]:
                if eta < eta_min:
                    epsilon += delta_epsilon
                    delta_epsilon /= 2
                    x_adv_hat = x_adv
                    break  
                eta /= 2
                x_adv_hat = torch.clamp(x_adv_hat, initial_image - epsilon, initial_image + epsilon)
            x_adv = x_adv_hat
            epsilon = epsilon - delta_epsilon
            target_class = torch.argmax(model(x_adv))
    return x_adv

In [5]:
test_img, test_label = testset[4]
test_img = test_img.unsqueeze(0)
initial_img, initial_label = testset[1]
initial_img = initial_img.unsqueeze(0)

In [8]:
reverse_mapping[test_label]

'CHECQUERED SKIPPER'

In [15]:
adv_img = PIA_adversarial_generator(model, initial_image = initial_img, 
                                    image = test_img, target_class = test_label,
                                    epsilon_adv = 0.05, epsilon_0 = 0.5,
                                    search_var = 1e-3, sample_num = 50,
                                    delta_epsilon = 1e-3, 
                                    eta_max = 0.01, eta_min = 0.005,
                                    k=5)

tensor([True], device='cuda:0')
20 times runned
tensor([[0.0093, 0.0119, 0.0056, 0.0112, 0.0089, 0.0031, 0.0192, 0.0045, 0.0052,
         0.0053, 0.0091, 0.0122, 0.0089, 0.0250, 0.0084, 0.0110, 0.0045, 0.0085,
         0.0116, 0.0049, 0.0097, 0.0058, 0.0154, 0.0195, 0.0094, 0.0074, 0.0096,
         0.0090, 0.0136, 0.0069, 0.0085, 0.0057, 0.0198, 0.0115, 0.0074, 0.0126,
         0.0101, 0.0055, 0.0058, 0.0065, 0.0083, 0.0228, 0.0044, 0.0118, 0.0067,
         0.0087, 0.0112, 0.0131, 0.0167, 0.0077, 0.0101, 0.0046, 0.0242, 0.0105,
         0.0050, 0.0076, 0.0068, 0.0044, 0.0086, 0.0067, 0.0110, 0.0128, 0.0093,
         0.0065, 0.0104, 0.0092, 0.0024, 0.0096, 0.0068, 0.0322, 0.0260, 0.0096,
         0.0054, 0.0131, 0.0176, 0.0036, 0.0017, 0.0199, 0.0048, 0.0053, 0.0040,
         0.0143, 0.0066, 0.0134, 0.0134, 0.0030, 0.0069, 0.0050, 0.0130, 0.0100,
         0.0147, 0.0137, 0.0188, 0.0085, 0.0075, 0.0034, 0.0111, 0.0087, 0.0105,
         0.0082]], device='cuda:0')
tensor(69, device='cuda:0

KeyboardInterrupt: 